## Import utils

In [0]:
%run ../utils/dataframe_utils

In [0]:
%run ../utils/file_utils

In [0]:
%run ../utils/table_utils

## Import libraries and define constants

In [0]:
import os
from pathlib import Path

# volume path contaning csv files
VOLUME_PATH = "/Volumes/workspace/default/file_ingestion"

## Clean file names and columns

In [0]:
files = os.listdir(VOLUME_PATH)

In [0]:
clean_dataframes = {}
raw_dataframes = {}

for file_name in files:
    try:
        file_path_obj = Path(VOLUME_PATH) / file_name
        file_name_clean = clean_file_name(file_path_obj.name)

        # Read CSV
        df = (
            spark.read.option("header", "true")
            .option("inferSchema", "false")
            .csv(str(file_path_obj))
        )

        df_cleaned = clean_column_names(df)

        clean_dataframes[file_name_clean] = df_cleaned
        raw_dataframes[file_name] = df

    except Exception as e:
        print(f"Error processing file {file_name}: {e}")

In [0]:
print(list(raw_dataframes.keys()))
display(list(raw_dataframes.values())[1])

In [0]:
print(list(clean_dataframes.keys()))
display(list(clean_dataframes.values())[1])

## Write tables to catalog

In [0]:
for table_name, df in clean_dataframes.items():
    table_config = DataProductConfig(
        table_name = table_name,
        schema = "world_telecom_suscriptions",
        layer = "bronze"
    )

    table_config.write_table(df)
    # df.write.mode("overwrite").saveAsTable(f"1_data_sources.world_telecom_suscriptions.{table_name}")